In [1]:
%run sources/sequence_dataset.py
%run sources/sequence_walking_tree.py

import random 

SAMPLES = 20

In [2]:
"""
from networkx.drawing.nx_pydot import to_pydot
from IPython.display import Image, display
import networkx as nx

def generate_tree(g, t, k=0):
    newline = "\n"
    g.add_node(k, label=f'{t["test_type"] if "test_type" in t else ""}{t["test"] if "test" in t else ""}{newline + "true_samples - "+str(t["true_samples"])+ " " if "true_samples" in t else "" }{newline+"false_samples - "+str(t["false_samples"]) if "false_samples" in t else "" }{newline+str(t["probabilities"])}'  )
    next_k = k + 1
    for key in [True, False]:
        if key in t:
            current_k, next_k = generate_tree(g, t[key], next_k)
            g.add_edge(k, current_k, label= f'{key}')
    return k, next_k

for i,t in enumerate(F.trees):
    G = nx.DiGraph()
    generate_tree(G,t.tree)
    pydot_graph = to_pydot(G)
    dot_file_path = f"tree{i}.dot"
    pydot_graph.write_dot(dot_file_path)
    !dot -Tpng tree{i}.dot -o tree{i}.png
    #display(Image(f"tree{i}.png"))
"""   

'\nfrom networkx.drawing.nx_pydot import to_pydot\nfrom IPython.display import Image, display\nimport networkx as nx\n\ndef generate_tree(g, t, k=0):\n    newline = "\n"\n    g.add_node(k, label=f\'{t["test_type"] if "test_type" in t else ""}{t["test"] if "test" in t else ""}{newline + "true_samples - "+str(t["true_samples"])+ " " if "true_samples" in t else "" }{newline+"false_samples - "+str(t["false_samples"]) if "false_samples" in t else "" }{newline+str(t["probabilities"])}\'  )\n    next_k = k + 1\n    for key in [True, False]:\n        if key in t:\n            current_k, next_k = generate_tree(g, t[key], next_k)\n            g.add_edge(k, current_k, label= f\'{key}\')\n    return k, next_k\n\nfor i,t in enumerate(F.trees):\n    G = nx.DiGraph()\n    generate_tree(G,t.tree)\n    pydot_graph = to_pydot(G)\n    dot_file_path = f"tree{i}.dot"\n    pydot_graph.write_dot(dot_file_path)\n    !dot -Tpng tree{i}.dot -o tree{i}.png\n    #display(Image(f"tree{i}.png"))\n'

In [3]:
def read_event_logs(csv_path):
    r = pd.read_csv(csv_path)
    return list(r["sequence"].apply(lambda x: eval(x) )), list(r["class"])

X,Y = read_event_logs("datasets/20240124_1525_fitbit_p02_quartiles.csv")

Z = SequenceDataset()
Z.fit(X[:SAMPLES],Y[:SAMPLES])

In [4]:
class SequenceWalkingForest():

    def __init__(self, number_of_trees = 10, tree_parameters = {}):
        self.number_of_trees = number_of_trees
        self.tree_parameters = tree_parameters
        self.trees        = []
        self.tree_weights = []
        self.tree_confusion_matrices = []

    def fit(self, z, weight_index = 0):
        z_size = len(z.raw_x)
        id_set = {i for i in range(z_size)}.difference({ i for i,_ in  z.excluded_positions(weight_index) })
        id_list = list(id_set)
        train_size = len(id_set)
        for i in range(self.number_of_trees):
            train_index = - (2*i + 1)
            score_index = - (2*i + 2)
            z.sample_df[train_index] = 0
            z.sample_df[score_index] = 0
            for i in range(len(z.sample_df)):
                if i not in id_set:
                    z.sample_df.loc[i, score_index] = 1
            for _ in range(train_size):
                j = random.choice(id_list)
                z.sample_df.loc[j, train_index] = z.sample_df.loc[j, train_index] + 1
                z.sample_df.loc[j, score_index] = 1
            self.trees.append(self.fit_tree(z, train_index, score_index))
            self.tree_weights.append(self.compute_weight(z,score_index))
            self.tree_confusion_matrices.append(self.compute_confusion_matrix(z,score_index))

    def fit_tree(self, z, train_index, score_index):
        r = SequenceWalkingTree(**self.tree_parameters)
        r.fit(z, train_index)
        z.predictions[score_index] = r.predict(z, score_index)
        self.compute_weight(z, score_index)
        return r
    
    def class_prediction_merge(self, z, score_index):
        return pd.merge(z.sample_df[['class']], z.predictions[score_index][['prediction']], left_index=True, right_index=True, how='inner')

    def compute_weight(self, z, score_index):
        merged = self.class_prediction_merge(z, score_index)
        return (merged["class"] == merged["prediction"]).mean()
    
    def compute_confusion_matrix(self, z, score_index):
        c = list(Z.classes)
        c.sort()
        merged = self.class_prediction_merge(z, score_index)
        r = pd.DataFrame([{ "prediction":  x, "class": y, "count": 0  } for x in c for y in c  ])
        r.set_index(["prediction", "class"], inplace=True,drop=True)
        for i in range(len(merged)):
            x, y = merged.iloc[i]["prediction"], merged.iloc[i]["class"]
            r.loc[(x,y), "count"] += 1
        return r
    
    def predict(self,z, weight_index = 0, start_index=1, include_class = True, keep_predictions = False):
        all_predictions = [] 
        for i in range(len(self.trees)):
            z.sample_df[start_index + i] = z.sample_df[weight_index]
            z.predictions[start_index + i] = self.trees[i].predict(z, start_index + i)
            if i == 0:
                all_predictions = z.predictions[start_index + i][["prediction"]]
            else:
                all_predictions = pd.merge(all_predictions, z.predictions[start_index + i][["prediction"]], suffixes=(f"_{i-1}", f"_{i}"), left_index=True, right_index=True, how="inner")
         
        voting = pd.merge(SequenceWalkingForest.majority(all_predictions), self.weighted(all_predictions), left_index=True, right_index=True, how='inner')
        voting = pd.merge(voting, self.track_record(all_predictions), left_index=True, right_index=True, how='inner')
        if keep_predictions:
           voting = pd.merge(voting, all_predictions, left_index=True, right_index=True, how='inner' )
        if include_class:
            excluded_samples = {i for i, _ in z.excluded_positions(weight_index)}
            voting = pd.merge(z.sample_df.loc[excluded_samples]["class"],voting, left_index=True, right_index=True, how='inner')
        z.forest_prediction[weight_index] = voting
        return voting   

    @staticmethod
    def majority(all_predictions):   
        r = all_predictions.apply(lambda row: pd.Series(list(row)).value_counts().idxmax(), axis=1) 
        r.name = "majority"
        return r   

    def best_weight(self, list):
        w = {c:0 for c in set(list)}
        for i, c in enumerate(list):
            w[c] = w[c] + self.tree_weights[i]
        return max(w,key=lambda k: w[k])

    def weighted(self, all_predictions):
        r = all_predictions.apply(lambda row: self.best_weight(list(row)), axis=1)
        r.name = "weighted"
        return r
    
    def best_track_record(self,list):
        w = {c:0 for c in set(list)}
        for i, c in enumerate(list):
            col = self.tree_confusion_matrices[i].loc[c]["count"]
            if col.sum() > 0:
                probability = col/col.sum()
                for k in w.keys():
                    w[k] = w[k] + probability.loc[k]
        return max(w,key=lambda k: w[k])    
            
    def track_record(self, all_predictions):
        r = all_predictions.apply(lambda row: self.best_track_record(list(row)), axis=1)
        r.name = "track_record"
        return r


In [5]:
Z.sample_df.loc[15:,0 ] = 0
Z.predictions = {}

In [6]:
F = SequenceWalkingForest(number_of_trees=20)
F.fit(Z,)

In [7]:
F.predict(Z,weight_index=0)

/var/folders/qv/_mh728qx5t16xhvl4mmhwzr40000gn/T/ipykernel_88863/3300932744.py:72: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  voting = pd.merge(z.sample_df.loc[excluded_samples]["class"],voting, left_index=True, right_index=True, how='inner')


,class,majority,weighted,track_record
sample,,,,
15,0,0,0,0
16,2,1,1,3
17,3,1,1,1
18,0,3,1,1
19,3,3,3,3


	
sample class	majority	weighted	track_record
15	0	0	0	0
16	2	1	1	3
17	3	0	1	1
18	0	3	2	1
19	3	3	3	3